In [ ]:
#install necessary packages

%pip install matplotlib pandas numpy scipy seaborn mne
%pip install beautifulsoup4 requests wget
%pip install h5py tables kaggle
%pip install wfdb pyEDFlib PyWavelets
%pip install keras
%pip install tensorflow

In [1]:
import numpy as np
import pandas as pd

#Import data into panda data frame
data = pd.read_csv(r'C:\Users\shortallb\Documents\Research\Epileptic Seizure Data - Kaggle\data.csv')
n = data.shape[0]
#Remove patient name 
data = data.drop(data.columns[0], axis =1)
#Extract Labels
labels = data.iloc[:,178].to_numpy()
data = data.drop(data.columns[178], axis =1)

#Reduce num classes from 5 to 2. (binary classifier)
labels_bin = np.zeros(n)
for i in range(n):
    if labels[i] == 1:
        labels_bin[i] = 1
       
#Import "Real" data from .CSV 
b1 = pd.read_csv('./baseline.csv', header=None).to_numpy()
s1 = pd.read_csv('./seizure1.csv', header=None).to_numpy()
s2 = pd.read_csv('./seizure2.csv', header=None).to_numpy()

#Convert to numpy array 
#data = data.to_numpy()


In [3]:
#Functions to compute mean band powers
from scipy.signal import welch
import numpy as np
sample_rate = 178 # in hz

data_test1 = data.iloc[0]    #Non-Seizure data
data_test2 = data.iloc[1]   #Seizure data

#Function gets the mean power of a specified frequency band. Will be used to calculate power estimations of most common frequency bands
def bandpower(data, sf, band, output = False):
    band = np.asarray(band)
    low, high = band

    # Compute the periodogram (Welch)
    freqs, psd = welch(data, 
                       sf, 
                       nperseg=(2 / low)*sf,
                       scaling='density', 
                       axis=0)
    
    # put into a df
    psd = pd.DataFrame(psd, index = freqs)
    
    if output:
        print('Welch Output')
        psd.index.name = 'Hz'
        psd.columns = ['Power']
        display(psd)
    
    # Find closest indices of band in frequency vector
    idx_min = np.argmax(np.round(freqs) > low) - 1
    idx_max = np.argmax(np.round(freqs) > high)
    
    # select frequencies of interest
    psd = psd.iloc[idx_min:idx_max,:]
    
    # get the mean of each channel over all frequencies in the band
    psd = psd.mean()
    
    if output:
        print('\nMean Frequency Band')
        display(psd)
    
    return psd

#Returns df of power densities for frequency bands
def power_measures(data, sample_rate, output=False):
    bandpasses = [[[0.1,4],'power_delta'],
                  [[4,8],'power_theta'],
                  [[8,12],'power_alpha'],
                  [[12,30],'power_beta'],
                  [[30,70],'power_gamma']
                 ]
    
    welch_df = pd.DataFrame()
    for bandpass, freq_name in bandpasses:
        bandpass_data = bandpower(data, sample_rate, bandpass)
        bandpass_data.index = [freq_name]
        
        if welch_df.empty:
            welch_df = bandpass_data

        else:
            welch_df = pd.concat([welch_df, bandpass_data])
        
    welch_df = welch_df.T
    
    if output:
        display(welch_df)
    
    return welch_df
#Code found from:     
#https://colab.research.google.com/github/Eldave93/Seizure-Detection-Tutorials/blob/master/02.%20Pre-Processing%20%26%20Feature%20Engineering.ipynb#scrollTo=Oe3h7w3tVbhK

In [4]:
#Convert signal data into power densities Col 0 - 5 : delta, theta, alpha, beta, gamma
power_data = np.empty((n,5))
sample_rate = 178 #Hz
for i in range(n):
    power_data[i] = power_measures(data.iloc[i], sample_rate, output=False).to_numpy()


c:\Users\shortallb\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 3560 is greater than input length  = 178, using nperseg = 178
  .format(nperseg, input_length))


In [5]:
data_test1 = data.iloc[0]
power_data = power_measures(data_test1, sample_rate, output=False).to_numpy()
power_sum = np.sum(power_data)
band_power_norm = power_data / power_sum


[0.39266835 0.10220203 0.2879403  0.17819973 0.03898958]
1539.9670415388803
[604.69632186 157.38776484 443.41857731 274.42171134  60.04266618]


In [4]:
from sklearn.model_selection import train_test_split

#split data into data and labels
x = np.concatenate((b1,s1,s2), axis = 0)
y = x[:,10000]
x = np.delete(x, 10000, axis=1)
#Shuffle and partion data into train and test. (20% total data used as test)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=.2, random_state=27)

#Reshape for LSTM input
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))


In [2]:
#https://ai-leader.com/2020/05/09/using-lstm-with-1d-2d-and-3d-array/
from keras import Model
from keras.layers import Input, Dense, Bidirectional
from keras.layers import LSTM
import tensorflow as tf
#construct callbacaks
#backup model following each epoch 
backup_callback = tf.keras.callbacks.BackupAndRestore(backup_dir="./tmp/backup")
#stop after accuracy becomes stagnant
stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy",
    min_delta=.0011,
    patience=2,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)
#CSV logger - outputs progress to csv file
csv_callback = tf.keras.callbacks.CSVLogger('./train_progress.csv', separator=",", append=True)

def define_module():
    #shape is hardcoded. change if need be
    input1= Input(shape=(10000, 1))
    lstm1 = Bidirectional(LSTM(units=32))(input1)
    dnn_hidden_layer1 = Dense(3, activation='relu')(lstm1)
    dnn_output = Dense(1, activation='sigmoid')(dnn_hidden_layer1)
    model = Model(inputs=[input1],outputs=[dnn_output])
    # compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model





In [7]:
#https://ieeexplore.ieee.org/document/9873913

#https://www.frontiersin.org/articles/10.3389/fncom.2021.650050/full
#Zscore normalization if we want to use EEG data and electrode data

#https://datascience.stackexchange.com/questions/33393/understanding-input-of-lstm
#how to format LSTM input

#https://keras.io/api/callbacks/tensorboard/
#callback functions
model.save('./LSTM1.hdf5')

In [ ]:
#run a single model using random state shuffle 
lstm2 = define_module()
lstm2.fit(X_train, Y_train, epochs=10, batch_size = X_train.shape[0], callbacks = [backup_callback, stop_callback, csv_callback])
lstm2.save('./LSTM2.hdf5')
scores = lstm2.evaluate(X_test, Y_test)

In [6]:
#kfold cross validation source: https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md
from sklearn.model_selection import KFold
#concatenate train and test data
inputs = np.concatenate(X_train, X_test)
labels = np.concatenate(Y_train, Y_test)

#define num folds and cross validator
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True)

accuracies = []
losses = []

# Construct and Evaluate models
fold_num = 1
for train, test in kfold.split(inputs, labels):
    model = define_module()
    #fit data
    history = model.fit(inputs[train], labels[train], batch_size = X_train.shape[0], epochs=10, callbacks = [backup_callback, stop_callback, csv_callback])
    #evaluate test data
    scores = model.evaluate(inputs[test], labels[test])
    #append accuracies/losses to array of all folds
    accuracies.append(scores[1])
    losses.append(scores[0])
    #increment fold num
    fold_num += 1


TypeError: only integer scalar arrays can be converted to a scalar index